In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pyproj
import xarray as xr # for reading netCDF files
from vis_tools import grab_gmt_boundaries # our custom module

###############################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '/opt/hysplit/18020300_ruapehu_12.5_0.01.nc'
###############################################

# projection stuff
WGS84 = pyproj.Proj('+init=epsg:4326')
NZTM = pyproj.Proj('+proj=tmerc +lat_0=0 +lon_0=173 +k=0.9996 +x_0=1600000 +y_0=10000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs')

model = xr.open_dataset(FILENAME)
src_loc = [-39.283333, 175.566667] # currently hard-coded for Ruapehu (need to add dict of volc locs)
src_E, src_N = pyproj.transform(WGS84, NZTM, src_loc[1], src_loc[0])

# coordinates -- spatial and temporal
lat = model['lat'].data
lon = model['lon'].data
model_E, model_N = pyproj.transform(WGS84, NZTM, lon, lat)

t = model['time'].data # hours

# ash thickness
td = model['total_deposition'].data # mm

In [ ]:
###########################################
# SPECIFY: model time step, plotting params
TIME_STEP = 1

ASH_MIN = 10**-2
ASH_MAX = 10**3
NUM_CON = 6
RES = 'f'
###########################################  
    
# grab the time step from the gridded data        
time_slice = td[TIME_STEP, :, :]

# replace all zero values with nan's
np.place(time_slice, time_slice==0, np.nan)

# tweak figure settings for readability
plt.figure(figsize=(14, 8))
mpl.rcParams.update({'font.size': 14})

# PLOT background geography
boundaries = grab_gmt_boundaries(-48,-33,165,180,RES) # HARD-CODED region for now
for b_lat, b_lon in zip(boundaries['latitude'], boundaries['longitude']):
    b_E, b_N = pyproj.transform(WGS84, NZTM, b_lon, b_lat)
    plt.plot(b_E, b_N, 'k', linewidth=0.75)

# PLOT ashfall (using log scale)
plt.pcolor(model_E, model_N, time_slice, cmap='hot_r', norm=colors.LogNorm(), vmin=ASH_MIN, vmax=ASH_MAX)
cb = plt.colorbar()
#plt.contour(model_E, model_N, time_slice, np.logspace(np.log10(ASH_MIN), np.log10(ASH_MAX), NUM_CON),\
#            colors='k', alpha = 0.5, linewidths=0.75)

# PLOT source location
plt.scatter(src_E, src_N, s=200, c='c', marker='*', edgecolors='k', linewidths=0.75, label='source')

# HARD-CODED plot limits for now
plt.xlim(1600000, 2100000)
plt.ylim(5600000, 6000000)
plt.gca().set_aspect('equal', adjustable='box')

plt.xlabel('easting')
plt.ylabel('northing')
plt.title(FILENAME.split('/')[-1][:-3] + '\n' + 'time step ' + str(TIME_STEP+1) + ' of 8')
cb.set_label('ash thickness (mm)')

plt.legend();